***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [6. Deconvolution in Imaging](6_0_introduction.ipynb)  
    * Previous: [6.2 Point Source Assumption (CLEAN)](6_2_clean.ipynb)  
    * Next: [6.4 Source Finding and Detection](6_4_source_finding.ipynb)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Import section specific modules:

In [ ]:
import matplotlib.image as mpimg
from IPython.display import Image
from astropy.io import fits
import aplpy

#Disable astropy/aplpy logging
import logging
logger0 = logging.getLogger('astropy')
logger0.setLevel(logging.CRITICAL)
logger1 = logging.getLogger('aplpy')
logger1.setLevel(logging.CRITICAL)

In [ ]:
from IPython.display import HTML
HTML('../style/code_toggle.html')

***

## 6.3 Residuals and Image Quality<a id='deconv:sec:iqa'></a>

Using `CLEAN` or another deconvolution methods produces 'nicer' images than the dirty image (except when deconvolution gets out of control). What an it means for an image to be 'nicer' is not a well defined metric, in fact it is almost completely undefined. When we talk of the quality of an image in sythesis imaging we rarely use a quantitative metric, but instead rely on the subjective opinion of the people looking at the image. I know, this is not very scientific. In fact it is a bit ridiculous that more work hasn't been done on to objectively define image quality in sythesis imaging. The field of computer vision has been around for decades, we just need a bit of effort to take some of the well-developed techniques from that field and incorporate them into radio astronomy. This is bound to happen as we have moved into a work of automated calibration, imaging, and deconvolution pipelines. It is just a matter of when. With that diatribe out of the way, let us discuss what is done to to assess the quality of a deconvolved image.

We have two some what related questions we need to answer when we are reducing visibilities into a final image:

* When should you halt the deconvolution process?
* What makes a good image?

In $\S$ 6.2 we covered how we can seperate out a sky model from noise using an iterative `CLEAN` deconvolution process. But, we did not discuss at what point we halt the process. There is no well-defined point in which to stop the process. Typically and *ad hoc* decision is made to run deconvolution for a fixed number of iterations or down to a certain flux level. These halting limits are set by adjusting the `CLEAN` parameters until a 'nice' image is produced. Or, if the visibilities have been flux calibrated, which is possible with some arrays, the signal is fixed to some real flux scale. Having knowledge about the array and observation a theoretical noise floor can be computed, then `CLEAN` can be ran to a known noise level. One could imagine there is a more automated way to decide when to halt `CLEAN`, perhaps keeping track of the iterations and deciding if there is a convergence?

As a thought experiment we can think about a observation with perfect calibration (we discuss calibration in Chapter 8, but for now it is sufficient to know that the examples we have been using have perfect calibration). When we run `CLEAN` on this observation, each iteration will transfer some flux from the residual image to the sky model. If we run this long enough then we will reach the observation noise floor. Then, we will start to deconvolved the noise from the image. And if you run this process for infinite iteration we will eventually have a sky model which contains all the flux, both from the sources and the noise. The residual image in this case will be empty. Now, this extreme case results in our sky model containing noise sources, this is not ideal. But, if we have not deconvoled enough flux then the sky model is incomplete and the residual image will contain PSF structure from the remaining flux. Thus, the challenge is to determine what is enough deconvolution to remove most of the true sky signal but not over-deconvolved such that noise is added to the sky model. As stated earlier, the typical way to do that at the moment is to do multiple deconvolutions, adjusting the parameters until a subjective solution is reached.

The second question of what makes a good image is why we still use subjective opinion. If we consider the realistic case of imaging and deconvolving a real set of visibilities then we have the added problem that there will be always be, at some level, calibration errors. These errors, and cause of these errors, can be identified by a trained eye whether it is poor gain calibration, interference, strong source sidelobes, or any number of other issues. Errors can cause a deconvolution process to diverge resulting in an unrealistic sky model. Humans are very good at looking at images and deciding if they make sense, but we can not easily describe how we do our image processing, thus we find it hard to implement algorithms to do the same. Looking at the dirty image and deconvolved image of the same field below most people would say the deconvoled image is objectively 'better' than the dirty image. Yet we do not know exactly why that is the case.

In [ ]:
fig = plt.figure(figsize=(16, 7))

gc1 = aplpy.FITSFigure('../data/fits/deconv/KAT-7_6h60s_dec-30_10MHz_10chans_uniform_n100-dirty.fits', \
                       figure=fig, subplot=[0.0,0.1,0.35,0.8])
gc1.show_colorscale(vmin=-1.5, vmax=3., cmap='viridis')
gc1.hide_axis_labels()
gc1.hide_tick_labels()
plt.title('Dirty Image')
gc1.add_colorbar()

gc2 = aplpy.FITSFigure('../data/fits/deconv/KAT-7_6h60s_dec-30_10MHz_10chans_uniform_n100-image.fits', \
                       figure=fig, subplot=[0.5,0.1,0.35,0.8])
gc2.show_colorscale(vmin=-1.5, vmax=3., cmap='viridis')
gc2.hide_axis_labels()
gc2.hide_tick_labels()
plt.title('Deconvolved Image')
gc2.add_colorbar()

fig.canvas.draw()

*Left: dirty image from a 6 hour KAT-7 observation at a declination of $-30^{\circ}$. Right: deconvolved image.*

The deconvolved image do not have the same noisey PSF structures around the sources that the dirty image does. We could say that these imaging artefacts are localized and related to the PSF response to bright sources. The aim of deconvolution is to remove these PSF like structures and replace them with a simple sky model which is decoupled fro the observing system. Most of difficult work in radio interferometry is the attempt to understand and remove the instrumentational effects in order to recover the sky signal. Thus, we have some context for why the deconvolved image is 'better' than the dirty image. The challenge in automatically answering what makes a good image is some how encoding both the context and human intution. Indeed, a challenge left to the reader.

### 6.3.1 Dynamic Range and Signal-to-Noise Ratio

**TODO: standard quality measurement (SNR, DR)**

* advantage is that it is easy to compute, disadvantage: says nothing about localized structure
* example: different SNR/DR based on sampling (random, all, noise from residual, region)

### 6.3.2 The Residual Image

In [ ]:
fig = plt.figure(figsize=(8, 7))

gc1 = aplpy.FITSFigure('../data/fits/deconv/KAT-7_6h60s_dec-30_10MHz_10chans_uniform_n100-residual.fits', \
                       figure=fig)
gc1.show_colorscale(vmin=-1.5, vmax=3., cmap='viridis')
gc1.hide_axis_labels()
gc1.hide_tick_labels()
plt.title('Residual Image')
gc1.add_colorbar()

fig.canvas.draw()

**TODO: residuals**

* residuals: what remains, what is the cause of the remaining structure (over/under clean, w-term, large scale structure of extended sources, poor clean settings)
* example: kat-7 deconvolution residuals

### 6.3.3 The Restored Image

In [ ]:
fig = plt.figure(figsize=(8, 7))

gc1 = aplpy.FITSFigure('../data/fits/deconv/KAT-7_6h60s_dec-30_10MHz_10chans_uniform_n100-image.fits', \
                       figure=fig)
gc1.show_colorscale(vmin=-1.5, vmax=3., cmap='viridis')
gc1.hide_axis_labels()
gc1.hide_tick_labels()
plt.title('Restored Image')
gc1.add_colorbar()

fig.canvas.draw()

**TODO: restored**

* building from the sky model and residuals, provides no additional information
* ideal beam
* example: restored image

### 6.3.4 Image Quality Assessment

**TODO**

* image quality assessment is really hard to automate because it is a complex opinion which is based on previous experience and is hard to reduce down to simple metrics, this needs more work
* once we are happy with the sky model and restored image, what now? we need to make a judgement call about what is a source, and what is noise, and if sky model components are localized enough to be considered part of the same source. for this we need source finding.

***

Next: [6.4 Source Finding and Detection](6_4_source_finding.ipynb)